# import the necessary packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import statistics as stats
import scipy
import string as st
import lmfit
from lmfit import Model
from lmfit import models
from lmfit.models import *
from scipy import io
from scipy.fftpack import dct, idct
from itertools import combinations
import tkinter as tk
from tkinter import filedialog
import os
import inspect
import csv

#saving each graph 
savefile = False

# define the likelihood ratio calculation function

Based on Aitken and Lucy (2004)  
https://doi.org/10.1046/j.0035-9254.2003.05271.x 

In [2]:
def likelihood_ratio(background, file1, file2):
    
    off_covar = None

    ######################################################
    ### some prior calculations on the background data ###
    ######################################################

    # Get the number of speakers
    num_speakers = len(background)

    # get the number of variables
    num_variables = len(next(iter(background.values())))

    # get the number of measurements 
    num_measur = 0
    for speaker in background.values():
        first_variable = next(iter(speaker.values()))
        num_measur += len(first_variable)

    # background mean : overall mean for each variable. 

    var_all = {}

    #first reshape the data to put all values together, regardless of speaker
    for speaker, variables in background.items(): 
        # loop through every variable 
        for variable, values in variables.items():
            # if the new variable has not yet been added to the new dictionary, add it.
            var_all.setdefault(variable, [])

            # add the values to the coefficient list
            var_all[variable].extend(values)

    #calculate the mean of each variable in the background data:

    background_means = {}

    for variable, values in var_all.items():
        background_means.setdefault(variable, )
        background_means[variable] = np.mean(values)

    #mean of each variable for eah of the speakers in the background data: 

    var_means_per_speaker = {}

    for speaker,var in background.items():
        var_means_per_speaker[speaker] = []
        for variable, values in background[speaker].items():
            var_means_per_speaker[speaker].append(np.mean(values))


    ######################################
    ### within group covariance matrix ###
    ######################################

    #calculated differently to make use of the cov function from the numpy package. 

    #create a covariance matrix for each speaker. 
                        
    cov_matrices = {}
    for speaker, values in background.items():
        speaker_data = np.vstack(list(values.values()))

        numeric_data = speaker_data.astype(np.number)
        cov_matrices[speaker] = np.cov(numeric_data)

    #adding all matrices

    matrices = [np.array(matrix) for matrix in cov_matrices.values()]
    sum_matrices = sum(matrix[:matrix.shape[0], :matrix.shape[1]] for matrix in matrices)

    # divide by the number of speakers.
    within_group = sum_matrices/num_speakers

    #######################################
    ### between group covariance matrix ###
    #######################################

    #according to the changes introduced by Morrison to accomodate for unequal n across speakers. 

    #calculate the first part of the group covariance matrix equation.
    Sstar= np.cov(np.array(list(var_means_per_speaker.values())), rowvar=False)

    #final calculations
    between_group_cov = Sstar - (sum_matrices/num_measur)

    #between group inverse covariance
    betw_inv = np.linalg.inv(between_group_cov)

    #between group square root
    betw_sqrt = scipy.linalg.sqrtm(between_group_cov)

    ####################################
    ### calulations on offender data ###
    ####################################

    # number of measures
    off_num_measures = len(next(iter(file1.values())))

    # offender mean
    off_mean = []
    for variable, values in file1.items():
        off_mean.append(stats.mean(file1[variable])) 

    #offender covariance matrix 

    off_covar= [[value / off_num_measures for value in row] for row in within_group]
        
    #offender inverse covariance matrix.
    off_inv = np.linalg.inv(off_covar)

    #offender square root 
    off_sqrt = scipy.linalg.sqrtm(off_covar)

    # offender square root inverse
    off_sqrt_inv = np.linalg.inv(off_sqrt)

     #################################
     ### suspect data calculations ###
    #################################

    # number of measures
    suspect_num_measures = len(next(iter(file2.values())))

    # suspect mean
    suspect_mean = []
    for variable, values in file2.items():
        suspect_mean.append(stats.mean(file2[variable])) 

    #suspect covariance matrix 

    suspect_covar = [[value / suspect_num_measures for value in row] for row in within_group]
    suspect_covar = np.array(suspect_covar)

    #suspect inverse covariance matrix.
    suspect_inv = np.linalg.inv(suspect_covar)

    #suspect square root 
    suspect_sqrt = scipy.linalg.sqrtm(suspect_covar)

    # suspect square root inverse
    suspect_sqrt_inv = np.linalg.inv(suspect_sqrt)

    ###########################
    ### smoothing parameter ###
    ###########################
    
    # as given by Silverman(1986)

    smooth_param= ((4/(2*num_variables+1))**(1/(num_variables+4)) 
                   * num_speakers**-(1/(num_variables+4)))

    ####################################
    ### a few other pre-calculations ###
    ####################################

    kernel = smooth_param**2 * between_group_cov

    inv_kernel = np.linalg.inv(kernel)

    kernel_typ = 0
    dist_back_off = 0
    dist_back_suspect = 0 
    suspect_off_mean_typ = 0

    suspect_off_mean_diff = np.subtract(suspect_mean, off_mean)
    suspect_off_mean_typ = np.linalg.solve(off_inv 
                                           + suspect_inv, np.linalg.solve(off_covar, off_mean) 
                                           + np.linalg.solve(suspect_covar, suspect_mean))
    
    for speaker, values in var_means_per_speaker.items():
        typicality = np.subtract(suspect_off_mean_typ, values)

        #kernel density at typicality.
        kernel_typ += (np.exp(-0.5 * typicality.T 
                             @ np.linalg.inv(np.linalg.inv(off_inv + suspect_inv) + kernel) 
                             @ typicality))

    #calculations of the distance between background and offender data
    dist_back_off += (np.exp(-0.5 * np.subtract(off_mean, values).T 
                                 @ np.linalg.inv(off_covar + kernel) 
                                 @ np.subtract(off_mean, values)))

    #calculations of the distance between background and suspect data
    dist_back_suspect += (np.exp(-0.5 * np.subtract(off_mean, values).T
                                 @ np.linalg.inv(off_covar + kernel) 
                                 @ np.subtract(off_mean, values)))

    #################
    ### numerator ###
    #################

    numerator = ((2 * np.pi) **(-num_variables) 
                     * np.linalg.det(off_sqrt_inv) 
                     * np.linalg.det(suspect_sqrt_inv)
                     * 1/np.sqrt(np.abs(np.linalg.det(between_group_cov))) 
                     * (num_speakers * smooth_param ** num_variables) ** (-1) 
                     * np.abs(np.linalg.det(off_inv + suspect_inv + inv_kernel)) ** (-0.5) 
                     * np.exp(-0.5 * 
                              suspect_off_mean_diff.T 
                              @ np.linalg.inv(suspect_covar + off_covar) 
                              @ suspect_off_mean_diff) 
                     * kernel_typ)

    ###################
    ### denominator ###
    ###################

    denominator = ( (2*np.pi)**-num_variables * 1/abs(np.linalg.det(between_group_cov))
                       * (num_speakers * smooth_param**num_variables)**-2 

                       * abs(np.linalg.det(suspect_sqrt_inv))
                       * abs(np.linalg.det(suspect_inv + inv_kernel))**-0.5
                       * dist_back_suspect

                       * abs(np.linalg.det(off_sqrt_inv))
                       * abs(np.linalg.det(off_inv + inv_kernel))**-0.5
                       * dist_back_off)

    ########################
    ### likelihood ratio ###
    ########################

    likelihood_ratio= numerator / denominator

    return(likelihood_ratio)


# upload and sort the data
first uploading the data (needs to be saved in the same folder as this notebook under the name 'formant_diphthongs.csv')

In [3]:
# Create the Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Open the file dialog window
input_file = filedialog.askopenfilename()

if not input_file.endswith(".csv"):
    print("Invalid file format. Please provide a CSV file.")

raw_df = pd.read_csv(input_file)
#print(raw_df)

The markers for the diphthongs are not Python-friendly and need to be changed. The new marker attempt to stay close to the german written form, alternatively, characters close to their IPA form can be chosen. The rest of the Python script will be using exclusively latin characters, to avoid confusion and problems with Python.\
The table gets then grouped by the column "file".

In [4]:
d={'\\ct\\at': 'or','e\\at': 'er',
   '\\ef\\at': 'e_r', 'e\\ri\\at': 'e_ra',
   'a\\at': 'ar', '\\e-h\\yc': 'ehu',
   '\\ef\\atje': 'e_rje', 'ah\\ef\\at': 'ahea',
   '\\hs\\at': 'ua', '\\e-je': 'eje',
   'i\\at': 'ir', '\\cti': 'oi',
   'a\\hs': 'au', '\\e-\\ri\\at': 'era',
   '\\yc\\at': 'u_a', 'e\\atje': 'erje',
   'ah\\at': 'ahr', '\\ef\\ar': 'e_a',
   'e\\ri\\e-': 'e_re', '\\efje': 'e_je',
   '\\e-j\\ef': 'eje_', '\\atje': 'rje'}
betterID = raw_df.replace(d)

#drop columns containing NaN values, prevent the code breaking if f1 is not calculated.
betterID = betterID.dropna(axis=1)
#print (betterID) 

filegroup = betterID.groupby(["file"])


# curve fitting

Each analysis iterates through the groups "file". Within each file, additional groups get created by diphthongs and can be filtered. This makes the analysis of different diphthong easier. Here only the diphthongs with the marker "ai" (as in "beide") is analysed.

The following cell uses a 3rd degree polynomial to fit the diphthongs. The structure functions as follow:
- a first iteration through each document (using 'filegroup')
- a grouping by diphthongs and the dropping of the first two columns. 
- a grouping by formants, using the columns names.
- the curve fitting of each line within each formant group. 

For testing, it is possible to prints out a representation of the raw data against the fitting. The graphic shows each fitted curve as a line of darker color and the raw data as lighter points. Each formant is also given a separate colour. 

In [5]:
#### calculating and displaying the curve fitting with polynomials 3rd degree: ####

### creating the result dict 
aip3res = {file: {'f1': {'c0': [], 'c1': [], 'c2': [], 'c3': []},
                  'f2': {'c0': [], 'c1': [], 'c2': [], 'c3': []},
                  'f3': {'c0': [], 'c1': [], 'c2': [], 'c3': []}}
           for file, _ in filegroup}

### some variables that need to be defined beforehand

### plot parameters ###
xaxis = np.arange(0,101,5)
x = xaxis
#plot = plt.figure()
#f1plot = plt.ylim(0,4000)

# regex search
aipattern = re.compile('ai')

 ##### p3 model and parameters #####
p3mod = PolynomialModel(3)
p3params = p3mod.make_params(c0=650, c1=1, c2=1, c3=1)

# Iterate over each file
for file, data in betterID.groupby(["file"]):
    # Get the group of the current file
    currentfile = filegroup.get_group((file))
    
    # Iterate over the diphthongs
    for dipht, data in currentfile.groupby(['diphthong']):
        # convert the diphthong label to a Regex string
        string = ''.join(dipht)
        # only work with the labels "ai"
        if aipattern.search(string):
            # Group the data of the current diphthong and drop the "file" and "diphthong" columns
            aidipht = data.drop(['file', 'diphthong'], axis=1).groupby(lambda x: re.search("f[1-3]", x).group(), axis=1)
            
            # Iterate over the 3 formants
            for formant in ['f1', 'f2', 'f3']:
                # Get the group of the current formant
                fgroup = aidipht.get_group((formant))
                
                # Iterate over rows of the current formant group
                for row, data in fgroup.iterrows():
                    # Get the values of the current row as a list and fit a model to them
                    f = fgroup.loc[row, :].values.flatten().tolist()
                    p3fit = p3mod.fit(f, p3params, x=xaxis)
                    
                    # Plot the best fit and the raw data points (best fit as darker line and raw data points as light dots)
                    #plot = plt.plot(x, p3fit.best_fit, '-', label=formant, color='red' if formant == 'f1' else 'b' if formant == 'f2' else 'orange')
                    #plt.plot(x, f, '.', label=formant, color='tomato' if formant == 'f1' else 'lightskyblue' if formant == 'f2' else 'khaki')
                    
                    # append the results to a dictionary
                    for c in range(4):
                        aip3res[file][formant][f'c{c}'].append(p3fit.params[f'c{c}'].value)
                        
    #plt.suptitle(file) 
    #plt.ylim(0,3400)
    if savefile == True :
        plt.savefig(file+'_005_f2_f3.pdf')
    #plt.show()


The following part modifies the structure of the dictionary and keeps only two levels: file and variables (instead of two levels for the variables).

In [6]:
#remove one level of variable from the dictionary.

# creating the new dictionary
speaker_dict = {}

for file, formants in aip3res.items():
    
    # if the speaker has not yet been added to the new dictionary, add it.
    speaker_dict.setdefault(file, {})
    
    # loop through every formant inside each recording (second level)   
    for formant_name, coeffs in formants.items():
        
        # loop through every coefficient inside each formant (third level)
        for coeff, values in coeffs.items():
            
            # create a new variable name in the form: formant_coefficient
            form_coeffs = formant_name + '_' + coeff
            # if the new variable has not yet been added to the new dictionary, add it.
            speaker_dict[file].setdefault(form_coeffs, [])
            
            # add the values to the coefficient list
            speaker_dict[file][form_coeffs].extend(values)

#print(speaker_dict)

# apply the MVKD function
The following chunk of code loops accross each possible pair from the list of available recordings. The MVKD function then is applied to each case. 

In [7]:
### attempt at groups (take two recordings out of the groups, loop through every combination)

c =list(combinations(speaker_dict,2)) # get all possible pairs form the list of all files. 
results = {}
pattern = r'([A-Za-z]+)(\d+)'

counter = 0 

for fil1, fil2 in c: # iterate through each pair of file 
    regex1 = re.compile(fil1) # compile into regex format
    regex2 = re.compile(fil2)
    background = {}
    file1 = {}
    file2 = {}
    background_sorted = {}
    for file in speaker_dict: # for each file in the result dict
        fil = ''.join(file) # compile file name for regex search 
        if regex1.search(fil): # if the current file correspond to file 1
            for variable, value in speaker_dict[fil].items():
                file1[variable] = speaker_dict[fil][variable]# append the results
        elif regex2.search(fil): # if the current file correspond to file 2
            for variable, value in speaker_dict[fil].items():
                file2[variable] = speaker_dict[fil][variable]
        elif regex1.search(fil) and regex2.search(fil): # file1 and file2 should always be different
            print("both files have the same name.")
        else: # the other files are added to the background data.
            background[fil] = speaker_dict[fil]
    
    # combine the recording per speaker, regardless of date. 
    
    for file, variables in background.items():
    # make sure that the names follow the naming convention (letters then numbers) 
        match = re.match(pattern, file)

        # otherwise raise an error
        if not match:
            print(file, 'does not follow the pattern letters+numbers')

        # get the name of the speaker (only the letters)
        speaker_name = match.group(1)
        
        background_sorted[speaker_name] = {}
        
        for variable, values in variables.items():

            # if the new variable has not yet been added to the new dictionary, add it.
            background_sorted[speaker_name].setdefault(variable, [])

            # add the values to the coefficient list
            background_sorted[speaker_name][variable].extend(values)

        # if the speaker has not yet been added to the new dictionary, add it.
        background_sorted.setdefault(speaker_name, {})
    
    #that's the level at which the sorting and the MVKD function will be used. 
    #print(fil1, fil2, ":\n", likelihood_ratio(background,file1,file2))
    
    pair = fil1 + "_" + fil2
    results[pair] = likelihood_ratio(background,file1,file2)

#print(results)

The likelihood ratios have been gathered in a dictionary and get saved in a csv table.

In [8]:
# Get the directory path of the script
script_dir = os.path.dirname(os.path.abspath(inspect.getframeinfo(inspect.currentframe()).filename))

# Specify the file path for saving the dictionary
file_path = os.path.join(script_dir, 'LRs_results.csv')

with open('LRs_results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for key, value in results.items():
        writer.writerow([key, value])